In [1115]:
#import standard data analysis libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [1116]:
df_players = pd.read_csv('df_players.csv')

In [1117]:
#make only for batting
df_players_batting = df_players[df_players['total_pa'] > 0]

In [1118]:
#selected features
current_batter_features = [
    'player_mlb_id',
    'year',
    'age',
    'years_after_28',
    'bats',
    'primary_position',
    'ops',
    'k_rate_batter',
    'bb_rate_batter',
    'batter_avg_exp_ba',
    'babip_batter',
    'fly_balls_ratio_batter',
    'ground_balls_ratio_batter',
    'line_drives_ratio_batter',
    'popups_ratio_batter',
    'hr_fb_pct_batter',
    'avg_lineup_position',
    'batter_avg_xwoba',
    'batter_avg_woba',
    'total_pa'
]

df_players_batting = df_players_batting[current_batter_features]

In [1119]:
df_players_batting.head()

,player_mlb_id,year,age,years_after_28,bats,primary_position,ops,k_rate_batter,bb_rate_batter,batter_avg_exp_ba,babip_batter,fly_balls_ratio_batter,ground_balls_ratio_batter,line_drives_ratio_batter,popups_ratio_batter,hr_fb_pct_batter,avg_lineup_position,batter_avg_xwoba,batter_avg_woba,total_pa
1,33438ca2fdf93ff73b2e1bcc5da3058a04cf1661,2021,26,0,R,p,0.666667,0.333333,0.000000,0.202500,0.500000,NaN,0.666667,NaN,NaN,NaN,10.500000,0.186000,0.300000,3
4,80f3a94c7bf1ed6397017744dbb75ff931902e2d,2021,34,6,R,1b,0.802607,0.217656,0.088280,0.343302,0.292804,0.188455,0.348048,0.159593,0.039049,0.270270,3.368421,0.405550,0.365902,657
6,b4246bd7233f08e749626eb0b7647d295f2e37b7,2021,24,0,R,rf,0.965172,0.237430,0.131285,0.403417,0.310881,0.218954,0.225490,0.205882,0.058824,0.358209,1.231707,0.519829,0.418627,358
8,81272c934b63edda7b2c6ca9f268f5a594ce5131,2021,26,0,R,ss,0.817233,0.281588,0.101083,0.339430,0.331230,0.247485,0.255533,0.148893,0.036217,0.203252,4.678832,0.408804,0.366065,554
13,c296dc19142d27a8a0d4a738256f7423945d04b6,2021,33,5,L,1b,0.425214,0.230769,0.076923,0.184407,0.222222,0.222222,0.388889,0.083333,0.055556,0.000000,9.090909,0.209815,0.224359,39


In [1120]:
#deal with n/a values here and in functions

#set infinite ratios to 1
df_players_batting.replace(np.inf, 1, inplace=True)

In [1121]:
#get leaguewide averages for stats
numeric_cols = df_players_batting.select_dtypes(include=np.number).columns.tolist()

stat_averages_league = df_players_batting.groupby('year')[numeric_cols].apply(
    lambda group: group.apply(
        lambda col: np.average(col.dropna(), weights=group.loc[col.dropna().index, 'total_pa']) 
        if col.name not in ['total_pa'] and group.loc[col.dropna().index, 'total_pa'].sum() > 0
        else col.mean(),  # Use col.mean() for total_pa column
        axis=0
    )
)

In [1122]:
#get averages by position
numeric_cols = df_players_batting.select_dtypes(include=np.number).columns.tolist()
numeric_cols += ['primary_position']

df_batting_num_cols = df_players_batting[numeric_cols]

#get means
stat_averages_position = df_batting_num_cols.groupby(['year', 'primary_position']).apply(
    lambda group: group.apply(
        lambda col: np.average(col.dropna(), weights=group.loc[col.dropna().index, 'total_pa']) 
        if col.name != 'total_pa' and group.loc[col.dropna().index, 'total_pa'].sum() > 0
        else col.mean(),  # Use col.mean() for total_pa column
        axis=0
    )
).fillna(0).reset_index()

In [1123]:
#replace stats for players with unknown positions with league averages
common_cols = stat_averages_league.columns.intersection(stat_averages_position.columns.drop('primary_position'))
for year in stat_averages_position['year'].unique():
    mask = (stat_averages_position['year'] == year) & (stat_averages_position['primary_position'] == 'Unknown')
    stat_averages_position.loc[mask, common_cols] = stat_averages_league.loc[
        stat_averages_league['year'] == year, common_cols
    ].values

In [1124]:
#check for nas
df_players_batting.isna().sum()

player_mlb_id                  0
year                           0
age                            0
years_after_28                 0
bats                           0
primary_position               0
ops                           12
k_rate_batter                  0
bb_rate_batter                 0
batter_avg_exp_ba            105
babip_batter                 104
fly_balls_ratio_batter       308
ground_balls_ratio_batter    143
line_drives_ratio_batter     286
popups_ratio_batter          454
hr_fb_pct_batter             308
avg_lineup_position            0
batter_avg_xwoba             105
batter_avg_woba                4
total_pa                       0
dtype: int64

In [1125]:
#these all make sense to set to 0
df_players_batting = df_players_batting.fillna(0)

In [1126]:
#put all of player's stats into one row
df_2021 = df_players_batting[df_players_batting['year'] == 2021]
df_2022 = df_players_batting[df_players_batting['year'] == 2022]
df_2023 = df_players_batting[df_players_batting['year'] == 2023][['player_mlb_id','total_pa']]
df_all = df_2021.merge(df_2022, on='player_mlb_id', how='outer', suffixes=("_2ya","_1ya")).drop(columns=['year_2ya','year_1ya'])
df_all = df_all.merge(df_2023, on='player_mlb_id',how='outer').drop(columns='player_mlb_id')

In [1127]:
#remove players who did not have a pa in 2021 or 2022
df_all = df_all[(df_all['total_pa_1ya'] > 0) | (df_all['total_pa_2ya'] > 0)]

#remove pitchers due to universal DH adoption (except if they got over 200 PAs in a season (Shohei))
df_all = df_all[((df_all['primary_position_1ya'] != 'p') & (df_all['primary_position_2ya'] != 'p')) | (df_all['total_pa_1ya'] > 200) | (df_all['total_pa_2ya'] > 200)]

In [1128]:
#check nulls
df_all.isna().sum()

age_2ya                          158
years_after_28_2ya               158
bats_2ya                         158
primary_position_2ya             158
ops_2ya                          158
k_rate_batter_2ya                158
bb_rate_batter_2ya               158
batter_avg_exp_ba_2ya            158
babip_batter_2ya                 158
fly_balls_ratio_batter_2ya       158
ground_balls_ratio_batter_2ya    158
line_drives_ratio_batter_2ya     158
popups_ratio_batter_2ya          158
hr_fb_pct_batter_2ya             158
avg_lineup_position_2ya          158
batter_avg_xwoba_2ya             158
batter_avg_woba_2ya              158
total_pa_2ya                     158
age_1ya                          106
years_after_28_1ya               106
bats_1ya                         106
primary_position_1ya             106
ops_1ya                          106
k_rate_batter_1ya                106
bb_rate_batter_1ya               106
batter_avg_exp_ba_1ya            106
babip_batter_1ya                 106
f

In [1129]:
#fill the non-numerical nulls
df_all['age'] = df_all['age_1ya'].fillna(df_all['age_2ya'] + 1)  # Fill missing age and adjust

df_all['years_after_28'] = df_all['years_after_28_1ya'].fillna(df_all['years_after_28_2ya'] + 1)  # Fill missing years_after_28 and adjust

df_all['primary_position'] = df_all['primary_position_1ya'].fillna(df_all['primary_position_2ya']).fillna('Unknown')

df_all['bats'] = df_all['bats_1ya'].fillna(df_all['bats_2ya'])

df_all = df_all.drop(columns=['age_1ya','age_2ya','years_after_28_1ya','years_after_28_2ya','bats_1ya','bats_2ya','primary_position_2ya','primary_position_1ya'])

#everything else can be filled with 0
df_all = df_all.fillna(0)

In [1130]:
df_all.columns

Index(['ops_2ya', 'k_rate_batter_2ya', 'bb_rate_batter_2ya',
       'batter_avg_exp_ba_2ya', 'babip_batter_2ya',
       'fly_balls_ratio_batter_2ya', 'ground_balls_ratio_batter_2ya',
       'line_drives_ratio_batter_2ya', 'popups_ratio_batter_2ya',
       'hr_fb_pct_batter_2ya', 'avg_lineup_position_2ya',
       'batter_avg_xwoba_2ya', 'batter_avg_woba_2ya', 'total_pa_2ya',
       'ops_1ya', 'k_rate_batter_1ya', 'bb_rate_batter_1ya',
       'batter_avg_exp_ba_1ya', 'babip_batter_1ya',
       'fly_balls_ratio_batter_1ya', 'ground_balls_ratio_batter_1ya',
       'line_drives_ratio_batter_1ya', 'popups_ratio_batter_1ya',
       'hr_fb_pct_batter_1ya', 'avg_lineup_position_1ya',
       'batter_avg_xwoba_1ya', 'batter_avg_woba_1ya', 'total_pa_1ya',
       'total_pa', 'age', 'years_after_28', 'primary_position', 'bats'],
      dtype='object')

In [1131]:
#get columns to be averaged
cols_1ya = ['ops_1ya', 'k_rate_batter_1ya', 
            'bb_rate_batter_1ya', 'batter_avg_exp_ba_1ya', 'babip_batter_1ya', 'batter_avg_xwoba_1ya', 'batter_avg_woba_1ya',
            'fly_balls_ratio_batter_1ya', 'ground_balls_ratio_batter_1ya', 
            'line_drives_ratio_batter_1ya', 'popups_ratio_batter_1ya', 
            'hr_fb_pct_batter_1ya', 'avg_lineup_position_1ya']

cols_2ya = ['ops_2ya', 'k_rate_batter_2ya', 
            'bb_rate_batter_2ya', 'batter_avg_exp_ba_2ya', 'babip_batter_2ya', 'batter_avg_xwoba_2ya', 'batter_avg_woba_2ya',
            'fly_balls_ratio_batter_2ya', 'ground_balls_ratio_batter_2ya', 
            'line_drives_ratio_batter_2ya', 'popups_ratio_batter_2ya', 
            'hr_fb_pct_batter_2ya', 'avg_lineup_position_2ya']

pa_1ya = 'total_pa_1ya'
pa_2ya = 'total_pa_2ya'

# Calculate the weighted averages and replace the original columns
for col_1ya, col_2ya in zip(cols_1ya, cols_2ya):
    weighted_avg_col = col_1ya[:-3]  # Get the base column name without _1ya or _2ya
    df_all[weighted_avg_col] = (
        (5 * df_all[col_1ya] * df_all[pa_1ya] + 3 * df_all[col_2ya] * df_all[pa_2ya]) /
        (5 * df_all[pa_1ya] + 3 * df_all[pa_2ya])
    )

# Drop the original _1ya and _2ya columns
df_all_weighed = df_all.drop(columns=cols_1ya + cols_2ya)

#do the same for plate appearances
df_all_weighed['total_pa_'] = (5*df_all_weighed['total_pa_1ya'] + 3*df_all_weighed['total_pa_2ya']) / 8

df_all_weighed.drop(columns=['total_pa_1ya','total_pa_2ya'],inplace=True)

In [1132]:
df_all_weighed.head()

,total_pa,age,years_after_28,primary_position,bats,ops_,k_rate_batter_,bb_rate_batter_,batter_avg_exp_ba_,babip_batter_,batter_avg_xwoba_,batter_avg_woba_,fly_balls_ratio_batter_,ground_balls_ratio_batter_,line_drives_ratio_batter_,popups_ratio_batter_,hr_fb_pct_batter_,avg_lineup_position_,total_pa_
0,60.0,24.0,0.0,lf,R,0.606482,0.309392,0.038674,0.232069,0.382609,0.243009,0.282402,0.127907,0.395349,0.098837,0.052326,0.045455,8.843137,113.125
1,91.0,27.0,0.0,lf,R,0.610894,0.184188,0.038688,0.317203,0.277229,0.338155,0.287928,0.162649,0.374018,0.209903,0.072094,0.122943,6.264132,148.625
2,44.0,22.0,0.0,c,R,0.311111,0.363636,0.090909,0.198083,0.166667,0.195917,0.152381,0.166667,0.388889,0.055556,0.055556,0.000000,9.285714,13.750
3,60.0,23.0,0.0,lf,R,0.281818,0.272727,0.090909,0.302000,0.142857,0.356286,0.186364,0.100000,0.450000,0.100000,0.050000,0.000000,7.000000,13.750
4,555.0,27.0,0.0,cf,L,0.736654,0.146828,0.079829,0.254249,0.253252,0.280376,0.327642,0.267959,0.287767,0.171879,0.103530,0.127653,4.462937,175.375


In [1133]:
df_all_weighed.select_dtypes(include=np.number).corr()

,total_pa,age,years_after_28,ops_,k_rate_batter_,bb_rate_batter_,batter_avg_exp_ba_,babip_batter_,batter_avg_xwoba_,batter_avg_woba_,fly_balls_ratio_batter_,ground_balls_ratio_batter_,line_drives_ratio_batter_,popups_ratio_batter_,hr_fb_pct_batter_,avg_lineup_position_,total_pa_
total_pa,1.000000,-0.159104,-0.152845,0.468975,-0.278954,0.108904,0.290034,0.215953,0.365040,0.450976,0.205146,0.029842,0.203600,0.019605,0.333440,-0.645516,0.694753
age,-0.159104,1.000000,0.913168,0.002991,-0.100000,0.017983,0.004872,-0.067482,0.004756,0.002209,0.060753,0.039877,0.030224,0.048288,-0.016584,-0.090173,0.180520
years_after_28,-0.152845,0.913168,1.000000,-0.010775,-0.108347,0.008403,-0.009509,-0.070833,-0.013152,-0.012209,0.046011,0.074266,0.020275,0.031033,-0.024134,-0.056079,0.090358
ops_,0.468975,0.002991,-0.010775,1.000000,-0.478757,0.251196,0.637630,0.713949,0.663430,0.981631,0.222756,0.007059,0.547799,0.012027,0.512931,-0.506697,0.500278
k_rate_batter_,-0.278954,-0.100000,-0.108347,-0.478757,1.000000,-0.085672,-0.190747,-0.197103,-0.106542,-0.506608,-0.293772,-0.563336,-0.522991,-0.179302,0.014425,0.340261,-0.358459
bb_rate_batter_,0.108904,0.017983,0.008403,0.251196,-0.085672,1.000000,0.253865,0.116822,0.261500,0.336307,-0.046469,-0.083871,0.022164,0.072872,0.156172,-0.144381,0.101415
batter_avg_exp_ba_,0.290034,0.004872,-0.009509,0.637630,-0.190747,0.253865,1.000000,0.605359,0.929008,0.637330,0.013647,-0.098406,0.483915,-0.174248,0.407934,-0.361441,0.323918
babip_batter_,0.215953,-0.067482,-0.070833,0.713949,-0.197103,0.116822,0.605359,1.000000,0.475095,0.716231,-0.041283,-0.005563,0.402871,-0.104544,0.137681,-0.205778,0.192547
batter_avg_xwoba_,0.365040,0.004756,-0.013152,0.663430,-0.106542,0.261500,0.929008,0.475095,1.000000,0.644760,0.134905,-0.204004,0.352462,-0.143843,0.602103,-0.437753,0.390841
batter_avg_woba_,0.450976,0.002209,-0.012209,0.981631,-0.506608,0.336307,0.637330,0.716231,0.644760,1.000000,0.182357,0.028688,0.545551,0.023887,0.475339,-0.497199,0.482401


In [1134]:
df_all_weighed.columns

Index(['total_pa', 'age', 'years_after_28', 'primary_position', 'bats', 'ops_',
       'k_rate_batter_', 'bb_rate_batter_', 'batter_avg_exp_ba_',
       'babip_batter_', 'batter_avg_xwoba_', 'batter_avg_woba_',
       'fly_balls_ratio_batter_', 'ground_balls_ratio_batter_',
       'line_drives_ratio_batter_', 'popups_ratio_batter_',
       'hr_fb_pct_batter_', 'avg_lineup_position_', 'total_pa_'],
      dtype='object')

In [1135]:
#regress everyone's stats to the mean of their position group by 5 PA (to help with players who had very few plate appearances)
stats_to_regress = ['ops_', 'k_rate_batter_', 'bb_rate_batter_', 'batter_avg_exp_ba_', 'batter_avg_xwoba_', 'batter_avg_woba_',
       'babip_batter_', 'fly_balls_ratio_batter_',
       'ground_balls_ratio_batter_', 'line_drives_ratio_batter_',
       'popups_ratio_batter_', 'hr_fb_pct_batter_']

In [1136]:
stat_averages_position_2021_2022 = stat_averages_position.drop(columns='year').reset_index().iloc[0:20].groupby('primary_position').mean().reset_index()

In [1137]:
df_all_weighed_regress = df_all_weighed.merge(stat_averages_position_2021_2022, on='primary_position', suffixes=('', '_incorrect'))


In [1138]:
for i in stats_to_regress:
    j = i.rstrip('_')
    k = i + 'reg'
    df_all_weighed_regress[k] = (df_all_weighed_regress[i]*df_all_weighed_regress['total_pa_'] + df_all_weighed_regress[j]*5 ) / (df_all_weighed_regress['total_pa_']+5)
    df_all_weighed_regress.drop([i, j ], axis=1, inplace=True)

df_all_weighed_regress.drop(df_all_weighed_regress.filter(like='_incorrect').columns, axis=1, inplace=True)
df_all_weighed_regress.drop(columns='index', inplace=True)

In [1139]:
df_all_weighed_regress

,total_pa,age,years_after_28,primary_position,bats,avg_lineup_position_,total_pa_,avg_lineup_position,ops_reg,k_rate_batter_reg,bb_rate_batter_reg,batter_avg_exp_ba_reg,batter_avg_xwoba_reg,batter_avg_woba_reg,babip_batter_reg,fly_balls_ratio_batter_reg,ground_balls_ratio_batter_reg,line_drives_ratio_batter_reg,popups_ratio_batter_reg,hr_fb_pct_batter_reg
0,60.0,24.0,0.0,lf,R,8.843137,113.125,5.002858,0.611495,0.306126,0.040820,0.236106,0.248626,0.284468,0.379086,0.130485,0.392169,0.102150,0.052211,0.051069
1,91.0,27.0,0.0,lf,R,6.264132,148.625,5.002858,0.614605,0.185752,0.040338,0.317536,0.339377,0.289336,0.277950,0.163501,0.372267,0.208835,0.071363,0.124738
2,44.0,22.0,0.0,c,R,9.285714,13.750,7.024247,0.406717,0.330876,0.088016,0.229825,0.239929,0.193641,0.195225,0.174029,0.367604,0.086104,0.056082,0.046309
3,60.0,23.0,0.0,lf,R,7.000000,13.750,5.002858,0.399977,0.261927,0.090501,0.308785,0.361465,0.224990,0.184598,0.123686,0.415391,0.120560,0.049901,0.047494
4,555.0,27.0,0.0,cf,L,4.462937,175.375,5.906514,0.735271,0.149676,0.079778,0.255936,0.282485,0.327271,0.254360,0.265671,0.288625,0.171722,0.102094,0.128575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
740,380.0,22.0,0.0,c,R,8.739130,45.625,7.024247,0.726720,0.122547,0.057290,0.312062,0.319643,0.327245,0.342470,0.136740,0.474612,0.212722,0.044866,0.117289
741,6.0,23.0,0.0,cf,L,7.875000,15.625,5.906514,0.408911,0.272622,0.018906,0.253619,0.260570,0.189818,0.249339,0.075251,0.168174,0.191803,0.254971,0.039008
742,530.0,26.0,0.0,2b,R,2.377428,505.625,5.021789,0.736830,0.219724,0.088845,0.323581,0.360528,0.350623,0.314250,0.187316,0.312267,0.167653,0.054736,0.165097
743,199.0,22.0,0.0,c,R,7.400000,10.000,7.024247,0.663687,0.205261,0.110019,0.302916,0.315169,0.310702,0.333678,0.207616,0.341118,0.151942,0.066795,0.057886


In [1140]:
df_all_weighed_regress.to_csv('batter_features.csv')